Data import


In [ ]:
import csv    #for the cvs and txv files
import sqlite3 as slq   #for the db file
import pandas as pd    #Help visualize the data

Sample the dataset


In [ ]:
df_1 = pd.read_csv("data/tmdb.movies.csv.gz")
df_1.head()

In [ ]:
df_2 = pd.read_csv("data/tn.movie_budgets.csv.gz")
df_2.head()

https://docs.python.org/3/library/codecs.html#standard-encodings - Helped with finding valid encoding 

In [ ]:
df_3 = pd.read_csv('data/rt.reviews.tsv.gz',delimiter = '\t', encoding = 'cp437' )
df_3.head()

In [73]:
df_4 = pd.read_csv('data/rt.movie_info.tsv.gz',delimiter = '\t')
df_4.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [ ]:
df_5 = pd.read_csv('data/bom.movie_gross.csv.gz',delimiter = '\t')
df_5.head()

Deep look in tmdb movie file

In [ ]:
####   Data Cleaning    ###
print(df_1.info())    #Unknown column
df_1['Unnamed: 0']    #No use, index column
df_1.drop("Unnamed: 0", axis = 1, inplace= True)   #Remove the column
df_1.columns    #Confirm the drop

In [ ]:
###    Check missing values    ###
print (df_1.isna().sum())    #No NaN values
print (df_1.isnull().sum())    #No Null Values


In [104]:
print(df_1.columns)
df_1['genre_ids']    #What does these numbers mean? - 
df_1['id']    #Each movie has a unique id? 
df_1['original_language']    #Only english movies? - No, there are multiples
df_1['original_title']     #Title of the movie
df_1['popularity']    #Ranking out of ... 100 , Website looks like percentage - based on amount timed watched?
df_1.sort_values('popularity').tail()
#Release data, title are pretty obvious, may need to look in title vs original title
df_1.loc[df_1['vote_average'] == 0]  #Quite a few with not votes and therefore no vote average
#df_1['vote_average'].plot(kind = 'hist')    #- Suject to remove with no votes
#df_1.sort_values(by ='vote_count')

Index(['genre_ids', 'id', 'original_language', 'original_title', 'popularity',
       'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')


,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
2467,[],331998,en,Edie: Girl on Fire,0.600,2010-01-01,Edie: Girl on Fire,0.0,1
4078,"[28, 27, 878, 37]",57817,en,Red River,0.875,2011-05-05,Red River,0.0,1
4282,[35],41840,en,Tom Wilson: Bigger Than You,0.666,2011-12-09,Tom Wilson: Bigger Than You,0.0,1
5176,"[18, 9648, 53]",102678,en,Dead Write,0.600,2011-01-01,Dead Write,0.0,1
5177,[18],81267,hi,क्षय,0.600,2011-10-08,Corrode,0.0,1
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [ ]:
###    Remove dups    ###
df_1.duplicated().sum()    #There are duplicated movies? Multiple languages?
df_1[df_1.duplicated(keep = False)].sort_values(by = 'id')    #Looks like straight copies - Need to remove
df_1.drop_duplicates(inplace = True)    #Remove dups
df_1.duplicated().sum()   #Check again for dups
